In [1]:
import urllib
from dotenv import load_dotenv
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import os 
import base64
import json

load_dotenv()

True

In [2]:
#Selenium items
from selenium import webdriver #webdriver to control activities
from selenium.webdriver.common.by import By #types of navigation
from selenium.webdriver.support.ui import WebDriverWait #waiting
import time

def start_driver(runheadless = False):
    """Initialize a Chrome webdriver with which for Selenium to act. """
    #set Chromedriver options to work smoothly
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized") # open Browser in maximized mode
    options.add_argument("disable-infobars") # disabling infobars
    options.add_argument("--disable-extensions") # disabling extensions
    options.add_argument("--disable-gpu") # applicable to windows os only
    options.add_argument("--disable-dev-shm-usage") # overcome limited resource problems
    options.add_argument("--no-sandbox") # Bypass OS security model  
    if runheadless:
        options.add_argument("--headless")  
    options.add_argument("--remote-debugging-port=9222")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument("--test-type")
    driver = webdriver.Chrome(options=options,
            executable_path=r"D:\chromedriver.exe")
    driver.implicitly_wait(10) # seconds
    
    driver.get("https://www.google.com/") #open the given URL
    time.sleep(1) #wait for everything to load

    return driver

driver = start_driver()

In [3]:
movie_list = pd.read_csv("1990_2020_movie_list1227.csv", encoding='latin1')
ogcols = ["title", "movieid", "year"]
ml_ls = movie_list[ogcols].values.tolist()
ml_ls[:5]

[['9', 'tt0472033', 2009],
 ['21', 'tt0478087', 2008],
 ['42', 'tt0453562', 2013],
 ['54', 'tt0120577', 1998],
 ['300', 'tt0416449', 2007]]

In [22]:
url_base = r"http://www.allmusic.com/search/albums/"
#ml = ml_ls[80]
store_dicts = []

for ml in ml_ls:
    search_me = "+".join([ml[0], "soundtrack", "(", str(ml[-1]), ")"])
    q = url_base + urllib.parse.quote(search_me)
    driver.get(q)
    time.sleep(1)
    content = driver.page_source
    soup = BeautifulSoup(content, "html")
    items = soup.find("div", id="cmn_wrap") #all("li", _class="album")
    items = items.find_all("li")
    its = [x for x in items if x.find("h4")]
    for i in its[:3]:
        dd = dict(zip(ogcols, ml))
        t = i.find_all("div")[1]
        ss = list(t.stripped_strings)
        dd['am_id'] = t.a['data-tooltip']
        dd['am_href'] = t.a['href']
        dd['am_title'] = ss[1]
        dd['am_year'] = ss[-2]
        dd['am_genres'] = ss[-1]
        if 'am_composer' in dd.keys():
            dd['am_composer'] = ss[2]
        store_dicts += [dd]

In [23]:
allmusicresults = pd.DataFrame(store_dicts)
allmusicresults.to_csv("allmusic_search_results.csv")

In [24]:
driver.quit()

In [48]:
albumlist = pd.read_csv(r"allmusic_search_validated.csv")
hrefs = albumlist.to_dict('records')
hrefs[:3]

[{'title': 'Josie and the Pussycats',
  'movieid': 'tt0236348',
  'year': 2001,
  'perf_match': 'mw0000000996',
  'am_href': 'https://www.allmusic.com/album/josie-and-the-pussycats-original-soundtrack-mw0000000996',
  'am_title': 'Josie and the Pussycats [Original Soundtrack]',
  'am_genres': 'Stage & Screen, Pop/Rock'},
 {'title': 'Along Came a Spider',
  'movieid': 'tt0164334',
  'year': 2001,
  'perf_match': 'mw0000001648',
  'am_href': 'https://www.allmusic.com/album/along-came-a-spider-mw0000001648',
  'am_title': 'Along Came a Spider',
  'am_genres': 'Stage & Screen'},
 {'title': "One Night at McCool's",
  'movieid': 'tt0203755',
  'year': 2001,
  'perf_match': 'mw0000001717',
  'am_href': 'https://www.allmusic.com/album/one-night-at-mccools-mw0000001717',
  'am_title': "One Night at McCool's",
  'am_genres': 'Stage & Screen, Pop/Rock'}]

In [69]:
#for h in hrefs 

#driver = start_driver()

h = hrefs[0]
print(h)

#driver.get(h)
#time.sleep(1)

#content = driver.page_source   

page = requests.get(h['am_href']).text
soup = BeautifulSoup(content, "html.parser")
side = soup.find("section", attrs={'class':'basic-info'})

for s in side.find_all("div"):
    ss = s.stripped_strings
    if ss:
        h[ss[0]] = ss[1]

{'title': 'Josie and the Pussycats',
 'movieid': 'tt0236348',
 'year': 2001,
 'perf_match': 'mw0000000996',
 'am_href': 'https://www.allmusic.com/album/josie-and-the-pussycats-original-soundtrack-mw0000000996',
 'am_title': 'Josie and the Pussycats [Original Soundtrack]',
 'am_genres': 'Stage & Screen, Pop/Rock',
 'Stream or buy on:': [],
 'Release Date': ['March 27, 2001'],
 'Duration': ['37:35'],
 'Genre': ['Stage & Screen', 'Pop/Rock'],
 'Stage & Screen': ['Pop/Rock'],
 'Styles': ['Soundtracks',
  'Alternative Pop/Rock',
  'Alternative/Indie Rock',
  'Pop Punk',
  'Teen Pop'],
 'Soundtracks': ['Alternative Pop/Rock',
  'Alternative/Indie Rock',
  'Pop Punk',
  'Teen Pop'],
 'Recording Location': ["Brandon's Way, Hollywood, CA",
  'Q Division, Somerville, MA',
  'Right Track Studios, New York, NY',
  'Tracken Place, Los Angeles, CA']}

<section class="basic-info">
<div class="streams num-2">
<h4>Stream or buy on:</h4>
<ul class="streaming-buttons clearfix">
<li>
<a class="discogs" data-target="discogs" href="https://www.discogs.com/sell/release/8123946" title="Find on Discogs"></a>
</li>
<li class="amazontooltip">
<a class="amazon" data-target="amazon" href='https://www.amazon.com/s/?field-keywords=music: "Josie%20and%20the%20Pussycats%20%5BOriginal%20Soundtrack%5D" by "Original%20Soundtrack"&amp;tag=allmusicdotcom-20' title="Search on Amazon"></a>
</li>
</ul>
</div>
<div class="release-date">
<h4>Release Date</h4>
<span>March 27, 2001</span>
</div>
<div class="duration">
<h4>Duration</h4>
<span>37:35</span>
</div>
<div class="genre">
<h4>Genre</h4>
<div>
<a href="https://www.allmusic.com/genre/stage-screen-ma0000004432">Stage &amp; Screen</a> <a href="https://www.allmusic.com/genre/pop-rock-ma0000002613">Pop/Rock</a> </div>
</div>
<div class="styles">
<h4>Styles</h4>
<div>
<a href="https://www.allmusic.com/style/sou

In [51]:
driver.quit()